In [ ]:
!nvidia-smi

Fri Jan  8 09:55:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#loading all the libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader

from PIL import Image, ImageDraw
import random

import os
import json

In [ ]:
image_path="/content/drive/MyDrive/Datasets/cassava-leaf-disease-classification"

#mapping the class names with labels
with open(os.path.join(image_path, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    map_classes = {int(k) : v for k, v in map_classes.items()}
    
print(json.dumps(map_classes, indent=4))

{
    "0": "Cassava Bacterial Blight (CBB)",
    "1": "Cassava Brown Streak Disease (CBSD)",
    "2": "Cassava Green Mottle (CGM)",
    "3": "Cassava Mosaic Disease (CMD)",
    "4": "Healthy"
}


In [ ]:
#Split the data into train, valid, test
df_data = pd.read_csv(os.path.join(image_path, "train.csv"))
df_data["class_name"] = df_data["label"].map(map_classes)

df  = pd.read_csv(image_path+"/train.csv")

df["path"] = df['image_id'].map(lambda x: image_path+"/train_images/"+x)
df = df.drop(columns=['image_id'])
df= df.sample(frac=1).reset_index(drop=True) #for random sample assignment

train = int(len(df) * 0.7) #70%
valid = int(len(df) * 0.2) #20%
test = len(df)-train-valid #10%

In [ ]:
train_df = df[:train]
valid_df = df[train + 1:valid+train].reset_index().drop(columns = ['index'])
test_df = df[train+valid+1:].reset_index().drop(columns = ['index'])

In [ ]:
#defining the transformation of the data
transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                transforms.CenterCrop((256,256)),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomVerticalFlip(p=0.5)
                                ])

In [ ]:
#defining the Preprossesing the data into image and label
class CassavaDataset(Dataset):
    def __init__(self, dataframe, transform = None):
        super().__init__()
        self.df = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.df["path"])
    
    def __getitem__(self, index):
        # get path and label
        path = self.df["path"][index]
        label = self.df["label"][index]
        # load image
        with open(path, 'rb') as f:
            image = Image.open(f)
            #convert it into RGB
            image = image.convert("RGB")
        # transform the image
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [ ]:
train_data = CassavaDataset(train_df, transform)
valid_data = CassavaDataset(valid_df, transform)
test_data = CassavaDataset(test_df, transform)

In [ ]:
train_data[0]

(tensor([[[-1.5357, -1.5357, -1.5528,  ..., -1.3130, -1.2617, -1.2103],
          [-1.5185, -1.5185, -1.5185,  ..., -1.2445, -1.2445, -1.2103],
          [-1.5185, -1.5014, -1.5014,  ..., -1.2103, -1.1760, -1.1589],
          ...,
          [-1.3473, -1.3815, -1.4158,  ..., -1.0219, -1.2617, -1.3130],
          [-1.3644, -1.4500, -1.4158,  ..., -1.1075, -1.3302, -1.3815],
          [-1.3302, -1.3644, -1.4500,  ..., -1.1075, -1.3473, -1.4158]],
 
         [[ 0.9580,  0.9580,  0.9755,  ..., -0.2325, -0.1275, -0.0574],
          [ 0.9755,  0.9930,  1.0105,  ..., -0.0924, -0.0574, -0.0049],
          [ 0.9755,  0.9930,  1.0280,  ..., -0.0049,  0.0301,  0.0476],
          ...,
          [ 0.6954,  0.6604,  0.6954,  ...,  1.2556,  1.0980,  1.0105],
          [ 0.6954,  0.6429,  0.6954,  ...,  1.1856,  1.0280,  0.9580],
          [ 0.7654,  0.7479,  0.6954,  ...,  1.1856,  1.0105,  0.9230]],
 
         [[ 0.7228,  0.7228,  0.7402,  ..., -1.2119, -1.3513, -1.4210],
          [ 0.7402,  0.7576,

In [ ]:
num_epoch = 10
num_classes = 5
batch_size = 64
lr = 0.001

In [ ]:
#Creating data loaders
train_loader = DataLoader(dataset = train_data, 
                          batch_size = batch_size,
                          shuffle=True,
                          num_workers = 0)

valid_loader = DataLoader(dataset = valid_data, 
                          batch_size = batch_size,
                          shuffle=False,
                          num_workers = 0)

test_loader = DataLoader(dataset = test_data, 
                          batch_size = batch_size,
                          shuffle=True,
                          num_workers = 0)

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#convolutional neural network
class Leaf_CNN(nn.Module):
    def __init__(self):
      super(Leaf_CNN, self).__init__()

      self.cnn_layers = nn.Sequential(
          
        #defining a 2D convolution layer 1
        nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),
    
        #defining a 2D convolution layer 2
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2), 

        #defining a 2D convolution layer 3
        nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),

        #defining a 2D convolution layer 4
        nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2,2),
        
      )

      self.linear_layers = nn.Sequential(
            nn.Linear(512, 256),
            nn.PReLU(),
            nn.Dropout(0.2, inplace=True),
            nn.Linear(256, 64),
            nn.PReLU(),
            nn.Dropout(0.2, inplace=True),
            nn.Linear(64, 5),
            ) 

    def forward(self, x):
        x = self.cnn_layers(x)
        x = torch.mean(x, dim = 3)
        x, _ = torch.max(x, dim = 2)
        x = self.linear_layers(x)
        return x

In [ ]:
model = Leaf_CNN().to(device)

criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = lr)

In [ ]:
#function for calculating the accuracy of the model
def calc_accuracy(model, df):
    model.eval()
    path = df['path']
    label = df['label']
    count = 0
    for i in range(len(path)):
        image_path = path[i]
        image_label = label[i]
        image = Image.open(image_path)
        image = transform(image)
        image = image.unsqueeze(0).to(device)
        model = model.to(device)
        pred = model(image).argmax(1).item()
        if pred == image_label:
            count += 1
    percent = count/len(path)
    return percent

In [ ]:
import time
def train_model(model, train_loader, valid_loader, optimizer, criterion, epoch):
    best_model = None
    best_loss = float("inf")
    # if "model.pth" not in output_list:
    train_losses, valid_losses = [], []
    acc = []
    for epoch in range(1, epoch + 1):
        epoch_start_time = time.time()
        train_loss = 0
        valid_loss = 0

        model.train()
        for data, target in train_loader:
            data = data.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            output = model(data)
            #for backward propagation
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*len(data)

        #updating the training loss
        train_loss = train_loss/len(train_loader.sampler)
        train_losses.append(train_loss)

        model.eval()
        for data, target in valid_loader:
            data = data.to(device)
            target = target.to(device)

            with torch.no_grad():
                output = model(data)
                pred = (output.argmax(1) == target)
                acc.append(sum(pred)/ len(pred))
                
                loss = criterion(output, target)
                
                valid_loss += loss.item()*len(data)
                if valid_loss < best_loss:
                    best_model = model
                    best_loss = valid_loss

        #Updating accuracy and validation loss
        collection = sum(acc)/len(acc)
        valid_loss = valid_loss/len(valid_loader.sampler)
        valid_losses.append(valid_loss)
        print('Time: {:.3f}\t Epoch: {} \tTraining Loss: {:.3f} \tValidation Loss: {:.6f} \t Acc: {:.2f}'
              .format(time.time() - epoch_start_time, epoch, train_loss, valid_loss, collection))
        num_collection = []
    torch.save(best_model.state_dict(), "./model.pth")
    
    return best_model

In [ ]:
best_model = train_model(model, train_loader, valid_loader, optim, criterion, num_epoch)

Time: 8736.145	 Epoch: 1 	Training Loss: 1.004 	Validation Loss: 1.200744 	 Acc: 0.57
Time: 512.358	 Epoch: 2 	Training Loss: 0.882 	Validation Loss: 1.361476 	 Acc: 0.61
Time: 503.401	 Epoch: 3 	Training Loss: 0.815 	Validation Loss: 0.881436 	 Acc: 0.63
Time: 505.876	 Epoch: 4 	Training Loss: 0.781 	Validation Loss: 0.855999 	 Acc: 0.65
Time: 497.903	 Epoch: 5 	Training Loss: 0.726 	Validation Loss: 0.946320 	 Acc: 0.66
Time: 494.087	 Epoch: 6 	Training Loss: 0.706 	Validation Loss: 0.699270 	 Acc: 0.67
Time: 498.044	 Epoch: 7 	Training Loss: 0.657 	Validation Loss: 0.789187 	 Acc: 0.68
Time: 488.407	 Epoch: 8 	Training Loss: 0.724 	Validation Loss: 0.710640 	 Acc: 0.69
Time: 489.352	 Epoch: 9 	Training Loss: 0.662 	Validation Loss: 0.803916 	 Acc: 0.69
Time: 485.247	 Epoch: 10 	Training Loss: 0.602 	Validation Loss: 0.610595 	 Acc: 0.70


In [ ]:
#accuracy on test data
calc_accuracy(best_model, test_df)

0.7542056074766356